In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import sys
sys.path.append("..")
from utils import *
from merge_utils import *
import re
import gc
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn import preprocessing
import pickle

In [2]:
df_train = pickle.load(open('./data/df_train_pca.pkl', 'rb'))
df_test = pickle.load(open('./data/df_test_pca.pkl', 'rb'))

In [16]:
df_train[cat_cols].count()/df_train.shape[0]

id_12            0.244239
id_13            0.215599
id_14            0.135544
id_15            0.238739
id_16            0.219020
id_17            0.236003
id_18            0.076393
id_19            0.235916
id_20            0.235820
id_21            0.008736
id_22            0.008753
id_23            0.008753
id_24            0.008038
id_25            0.008690
id_26            0.008743
id_27            0.008753
id_28            0.238727
id_29            0.238727
id_30            0.131346
id_31            0.237549
id_32            0.131381
id_33            0.124105
id_34            0.131752
id_35            0.238739
id_36            0.238739
id_37            0.238739
id_38            0.238739
M1               0.540929
M2               0.540929
M3               0.540929
M4               0.523412
M5               0.406506
M6               0.713212
M7               0.413647
M8               0.413669
M9               0.413669
card1            1.000000
card2            0.984873
card3       

In [5]:
df = df_train.append(df_test, sort=False)

In [7]:
del df_train, df_test

In [7]:
cat_cols = ['id_%i'%x for x in range(12, 39)] + ['M%i'%x for x in range(1, 10)] + ['card%i'%i for i in range(1,7)] + \
            [ 'DeviceType', 'DeviceInfo', 'ProductCD','P_emaildomain',
            'R_emaildomain', 'addr1', 'addr2', 'Card_ID', '_Hours', '_Weekdays']
num_cols = ['TransactionAmt','id_02'] + ['C%i'%i for i in range(1, 15)] + ['D%i'%i for i in range(1, 16)] 
# def brute(df):
#     for cat_col in cat_cols:
#         g = df.groupby(cat_col)
#         for num_col in num_cols:
#             df['%s#%s_mean1'%(num_col, cat_col)] = (df[num_col]-g[num_col].transform('mean'))
#             df['%s#%s_mean2'%(num_col, cat_col)] = df[num_col]/g[num_col].transform('mean')
#             df['%s#%s_mean3'%(num_col, cat_col)] = (df[num_col]-g[num_col].transform('mean'))/g[num_col].transform('std')
# brute(df)    

In [26]:
def split_gen(index, fold):
    num = len(index)//fold
    for i in range(fold-1):
        yield index[(num*i):(num*(i+1))]
    yield index[num*(fold-1):]

In [30]:
# df_train = df[df['label'].notnull()]
# df_test = df[df['label'].isnull()]
# del df
gc.collect()
for i, idx in enumerate(split_gen(df_train.index, 5)):
    with open('./data/df_train_agg_part%i.pkl'%i, 'wb') as f:
        pickle.dump(df_train.loc[idx, :], f)

# with open('./data/df_test_agg.pkl', 'wb') as f:
#     pickle.dump(df_test, f)

In [2]:
gc.collect()
# for i in range(5):
#     if i == 0:
#         df_test = pickle.load(open('./data/df_test_agg_part%i.pkl'%i, 'rb'))
#     else:
#         df_test = df_test.append(pickle.load(open('./data/df_test_agg_part%i.pkl'%i, 'rb')), sort=False)
#         gc.collect()
for i in range(5):
    if i == 0:
        df_train = pickle.load(open('./data/df_train_agg_part%i.pkl'%i, 'rb'))
    else:
        df_train = df_train.append(pickle.load(open('./data/df_train_agg_part%i.pkl'%i, 'rb')), sort=False)
        gc.collect()

In [3]:
def xgb_model_v3(params, df_train, df_val, features, label='label', num_boost_round=200):
    Dtrain = xgb.DMatrix(df_train[features], df_train[label])
    Dval = xgb.DMatrix(df_val[features], df_val[label])
    watchlist = [(Dtrain, 'train'), (Dval, 'val')]
    model = xgb.train(params, dtrain=Dtrain, early_stopping_rounds=100, 
                      evals=watchlist, num_boost_round=num_boost_round, verbose_eval=20)
    auc = model.best_score
    print('model best auc: %.4f at ntree_limit-%i'%(auc, model.best_ntree_limit))
#     Dtest = xgb.DMatrix(df_test[features])
#     pred = model.predict(Dtest, model.best_ntree_limit)
    return model, auc

In [4]:
tmp = ['id_%i'%x for x in range(12, 39)] + ['M%i'%x for x in range(1, 10)] + \
            [ 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6','P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2']
for col in tmp:
    if col in df_train.columns:
        le = preprocessing.LabelEncoder()
        le.fit(list(df_train[col].astype(str).values))
        df_train[col] = le.transform(list(df_train[col].astype(str).values))
        #df_test[col] = le.transform(list(df_test[col].astype(str).values))  
gc.collect()
params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eta': 0.05,   
    'gamma': 0.1,                  
    'max_depth': 20,  
    #'num_leaves': 12,
    'subsample': 0.7,  
    'colsample_bytree': 0.7,       
    'min_child_weight': 100,
    'reg_alpha': 5.,
    'reg_lambda': 5.,
    'scale_pos_weight': 27.6,               
    'eval_metric': ['auc'],
    'tree_method': 'gpu_hist',
}
folds = TimeSeriesSplit(n_splits=5)
f =[x for x in df_train.columns.drop(['label', 'TransactionDT', 'Date', 'Card_ID']) if 'mean' not in x]
to_drop =  ['_Days', 'Month']
f = list(set(f)-set(to_drop))
f += [x for x in df_train.columns if 'mean1' in x]
print(len(f))
auc_lst = []
for fold_n, (train_index, valid_index) in enumerate(folds.split(df_train)):
    if fold_n == 4:
        model, auc = xgb_model_v3(params, df_train.iloc[train_index, :], df_train.iloc[valid_index, :], f, 'label', num_boost_round=600)
    else:
        _, auc = xgb_model_v3(params, df_train.iloc[train_index, :], df_train.iloc[valid_index, :], f, 'label', num_boost_round=600)
    auc_lst.append(auc)
    gc.collect()
print(auc_lst)
print(np.mean(auc_lst))

1721
[0]	train-auc:0.917947	val-auc:0.809511
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 100 rounds.
[20]	train-auc:0.975844	val-auc:0.88076
[40]	train-auc:0.986033	val-auc:0.887817
[60]	train-auc:0.991661	val-auc:0.891377
[80]	train-auc:0.994879	val-auc:0.893662
[100]	train-auc:0.996573	val-auc:0.895288
[120]	train-auc:0.99765	val-auc:0.896242
[140]	train-auc:0.998405	val-auc:0.896444
[160]	train-auc:0.998831	val-auc:0.896222
[180]	train-auc:0.999139	val-auc:0.896586
[200]	train-auc:0.999323	val-auc:0.896561
[220]	train-auc:0.999454	val-auc:0.896262
Stopping. Best iteration:
[128]	train-auc:0.998025	val-auc:0.89697

model best auc: 0.8970 at ntree_limit-129
[0]	train-auc:0.926251	val-auc:0.825254
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 100 rounds.
[20]	train-auc:0.978978	val-auc:0.900472
[40]	train-auc:0.987142	val-

In [5]:
gain = pd.Series(model.get_score(importance_type='gain')).sort_values(ascending=False)
weight = pd.Series(model.get_score(importance_type='weight')).sort_values(ascending=False)
score = pd.concat([gain, weight], axis=1, sort=False)
score.columns = ['gain', 'weight']
score.to_csv('./result/agg_mean1_score.csv')

In [8]:
score

,gain,weight
C1#id_36_mean1,9965.307013,19
C1#DeviceType_mean1,7866.581849,125
C1#id_12_mean1,5830.244887,20
C14,3552.392367,48
D3#M4_mean1,3194.316880,60
C2#DeviceType_mean1,3081.618277,21
C14#card6_mean1,1706.237986,62
C14#id_36_mean1,1619.304519,21
C14#id_37_mean1,1323.999476,32
D2#id_34_mean1,1259.364304,4


In [10]:
tmp = pd.DataFrame(np.zeros(shape=(len(num_cols), len(cat_cols))), index=num_cols, columns=cat_cols)
#print(tmp)
for x in score.index:
    if '#' in x:
        num, cat = x.split('#')
        #print(num, cat)
        cat = cat[:-6]
        tmp.loc[num, cat] = score.loc[x, 'gain']
tmp.to_csv('./result/agg_gain_matrix_mean1.csv')

In [12]:
count(df_train, 'TransactionAmt_by_id_25_mean3', 10 )

(                                  total        1      rate
 TransactionAmt_by_id_25_mean3                             
 (-3.153, -0.84]                   489.0     27.0  0.055215
 (-0.84, -0.741]                   537.0     10.0  0.018622
 (-0.741, -0.582]                  468.0     26.0  0.055556
 (-0.582, -0.475]                  615.0     20.0  0.032520
 (-0.475, -0.209]                  429.0     36.0  0.083916
 (-0.209, 0.0]                     404.0     18.0  0.044554
 (0.0, 0.0565]                     493.0     10.0  0.020284
 (0.0565, 0.588]                   601.0     88.0  0.146423
 (0.588, 1.119]                    399.0     48.0  0.120301
 (1.119, 8.711]                    452.0    120.0  0.265487
 NaN                            585653.0  20260.0  0.034594,
 0.021627016416661978)

In [11]:
count(df_train, 'id_36')

(        total        1      rate
 id_36                           
 0      134066  10978.0  0.081885
 1        6919    244.0  0.035265
 2      449555   9441.0  0.021001, 0.4493387641586826)

In [ ]:
gc.collect()
params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eta': 0.04,   
    'gamma': 0.1,                  
    'max_depth': 18,  
    #'num_leaves': 12,
    'subsample': 0.7,  
    'colsample_bytree': 0.7,       
    'min_child_weight': 100,
    'reg_alpha': 1.,
    'reg_lambda': 5.,
    'scale_pos_weight': 27.6,               
    'eval_metric': ['auc'],
    'tree_method': 'gpu_hist',
}
folds = TimeSeriesSplit(n_splits=5)
f = f1
#f.remove('ProductCD')
print(len(f))
auc_lst = []
for fold_n, (train_index, valid_index) in enumerate(folds.split(df_train)):
    model, auc = xgb_model_v3(params, df_train.iloc[train_index, :], df_train.iloc[valid_index, :], f, 'label', num_boost_round=600)
    auc_lst.append(auc)
    gc.collect()
print(auc_lst)
print(np.mean(auc_lst))

2002


In [ ]:
gc.collect()
params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eta': 0.01,   
    'gamma': 0.2,                  
    'max_depth': 30,  
    #'num_leaves': 12,
    'subsample': 0.7,  
    'colsample_bytree': 0.6,       
    'min_child_weight': 100,
    'reg_alpha': 1.,
    'reg_lambda': 5.,
    'scale_pos_weight': 27.6,               
    'eval_metric': ['auc'],
    'tree_method': 'gpu_hist',
}
folds = TimeSeriesSplit(n_splits=5)
f = [x for x in df_train.columns if 'pca_0' in x] + [x for x in df_train.columns.drop(['label', 'TransactionDT', 'Date', 'Card_ID'])
                                                    if x[0]!='V' ]
to_drop =  ['_Days']
f = list(set(f)-set(to_drop))
#f.remove('ProductCD')
print(len(f))
auc_lst = []
for fold_n, (train_index, valid_index) in enumerate(folds.split(df_train)):
    if fold_n == 4:
        model, auc = xgb_model_v3(params, df_train.iloc[train_index, :], df_train.iloc[valid_index, :], f, 'label', num_boost_round=600)
    else:
        _, auc = xgb_model_v3(params, df_train.iloc[train_index, :], df_train.iloc[valid_index, :], f, 'label', num_boost_round=600)
    auc_lst.append(auc)
    gc.collect()
print(auc_lst)
print(np.mean(auc_lst))